In [1]:
import numpy as np
import xarray as xr

In [2]:
def Tb_to_OLR(Tb):
    a = 1.228
    b = -1.106e-3
    sigma = 5.67e-8  # W m^-2 K^-4
    OLR = ((((Tb * (2*b) + a)**2 - (a**2)) / (4*b))**4) * sigma
    return OLR


def OLR_to_TB(OLR):
    a = 1.228
    b = -1.106e-3
    sigma = 5.67e-8  # W m^-2 K^-4
    tf = (OLR/sigma)**0.25
    Tb = (-a + np.sqrt((a**2) + (4*b*tf)))/(2*b)
    return Tb

In [3]:
directory_sel = '/glade/derecho/scratch/molina/MCSMIP/mloutput/'
mcs_dir = '/glade/derecho/scratch/molina/MCSMIP/OLR_Precipitation_combined/'

# ---

season_select = 'Summer'
# season_select = 'Winter'
# model_select = 'UM'
# model_select = [
#     'ARPEGE', 'GEOS', 'GRIST', 'ICON', 'IFS',
#     'MPAS', 'SAM', 'SCREAM', 'UM', 'XSHiELD'
# ]
# model_select = [
#     'SCREAM', 'UM', 'XSHiELD'
# ]
model_select = 'OBS'
# model_select = 'OBSv7'
# model_select = ['SCREAMv1']

# ---

In [4]:
# only OBS (no other models)
filemcss = f"{directory_sel}/{season_select}/{model_select}_*.nc"
filename = f"olr_pcp_{season_select}_{model_select}.nc"

ds_ml = xr.open_mfdataset(filemcss)
ds = xr.open_dataset(mcs_dir+filename)

assert np.all(ds['time'] == ds_ml['time'])

ds = ds.assign(olr=Tb_to_OLR(ds['Tb']))
ds = ds.assign(mcs_prob=ds_ml['data'][:, 1, :, :])

savedir = f"/glade/derecho/scratch/molina/MCSMIP/mloutput/{ds.attrs['phase']}/"
ds.to_netcdf(f"{savedir}MCSMIP_{ds.attrs['phase']}_{ds.attrs['source']}.nc")
print(savedir)

/glade/derecho/scratch/molina/MCSMIP/mloutput/Summer/


In [4]:
# all models except OBS
for ms in model_select:

    filemcss = f"{directory_sel}/{season_select}/{ms}_*.nc"
    filename = f"olr_pcp_{season_select}_{ms}.nc"

    ds_ml = xr.open_mfdataset(filemcss)
    ds = xr.open_dataset(mcs_dir+filename)

    assert np.all(ds['time'] == ds_ml['time'])

    ds = ds.assign(Tb=OLR_to_TB(ds['olr']))
    ds = ds.assign(mcs_prob=ds_ml['data'][:, 1, :, :])

    savedir = f"/glade/derecho/scratch/molina/MCSMIP/mloutput/{ds.attrs['phase']}/"
    ds.to_netcdf(f"{savedir}MCSMIP_{ds.attrs['phase']}_{ds.attrs['source']}.nc")
    print(savedir)

/glade/derecho/scratch/molina/MCSMIP/mloutput/Winter/


### add attrs to created files

In [3]:
# summer_models = [
#     "ARPEGE.nc", "IFS.nc", "NICAM.nc", "SAM.nc", "FV3.nc", "MPAS.nc", "OBS.nc", "UM.nc"
# ]

# winter_models = [
#     "ARPEGE.nc", "ICON.nc", "OBS.nc", "UM.nc", "GEOS.nc", "IFS.nc", "SAM.nc", 
#     "XSHiELD.nc", "GRIST.nc", "MPAS.nc", "SCREAM.nc"
# ]

summer_models = [
    "OBSv7.nc", "OBS.nc"
]

winter_models = [
    "OBSv7.nc", "OBS.nc"
]

In [4]:
season = 'Summer'
directory = f"/glade/derecho/scratch/molina/MCSMIP/mloutput/{season}/MCSMIP_{season}_"
for i in summer_models:
    xr.open_dataset(directory+i).assign_attrs(
        author="Maria J. Molina",
        email="mjmolina@umd.edu",
        description="MCSs (mcs_prob) detected using deep learning model."
    ).to_netcdf(directory+"DL_"+i)

In [5]:
season = 'Winter'
directory = f"/glade/derecho/scratch/molina/MCSMIP/mloutput/{season}/MCSMIP_{season}_"
for i in winter_models:
    xr.open_dataset(directory+i).assign_attrs(
        author="Maria J. Molina",
        email="mjmolina@umd.edu",
        description="MCSs (mcs_prob) detected using deep learning model."
    ).to_netcdf(directory+"DL_"+i)